In [6]:
#Initializing an LLM and Setting up Langsmith

import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Langchain Tutorial"

from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-20b")

## Introduction to Tools
Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the @tool decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

* A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.

* Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.

* Both parameter and return type annotations.

In [7]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the @tool decorator our function is turned into a StructuredTool object, which we can see below:

In [8]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x00000217037D9260>)

In [9]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [10]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [11]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as kwargs (keyword arguments) as indicated by the ** operator - the ** operator is used to unpack the dictionary into keyword arguments.

In [12]:
exponentiate.func(**llm_output_dict)

25

### Creating an Agent

We're going to construct a simple tool calling agent. We will use LangChain Epression Language (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:

agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
We need this agent to remember previous interactions within the conversation. To do that, we will use the ChatPromptTemplate with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's "notes" as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a Helpful Assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),   
    ("placeholder", "{agent_scratchpad}"),
])

When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older ConversationBufferMemory class rather than the newer RunnableWithMessageHistory — the reason being that we will also be using the older create_tool_calling_agent and AgentExecutor method and class.

In the 05 chapter we will be using the newer RunnableWithMessageHistory class as we'll be building a custom AgentExecutor.

In [14]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages= True
)

C:\Users\Ankur Singh\AppData\Local\Temp\ipykernel_25560\800012107.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Now we will initialize our agent. For that we need:

* llm: as already defined <br>
* tools: to be defined (just a list of our previously defined tools) <br>
* prompt: as already defined <br>
* memory: as already defined <br>

In [15]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm = llm,
    tools = tools,
    prompt = prompt
)

Our agent by itself is like one-step of our agent execution loop. So, if we call the agent.invoke method it will get the LLM to generate a single response and go no further, so no tools will be executed, and no next iterations will be performed.

We can see this by asking a query that should trigger a tool call:

In [16]:
res = agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

In [17]:
from pprint import pprint
pprint(res)

[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to compute 10.7 * 7.68. We can do manually or use the multiply function. Use function.', 'tool_calls': [{'id': 'fc_84f16ed6-9ee7-4e0b-ad38-e3b5c88e9c69', 'function': {'arguments': '{"x":10.7,"y":7.68}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 229, 'total_tokens': 287, 'completion_time': 0.057699392, 'prompt_time': 0.011174739, 'queue_time': 0.042881261, 'total_time': 0.068874131}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3d587a02fb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2a4cbf10-8149-48ea-ac1d-f7d60ec46096-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': 'fc_84f16ed6-9ee7-4e0b-ad38-e3b5c88e9c69'

Here, we can see the LLM has generated that we should use the multiply tool and the tool input should be {"x": 10.7, "y": 7.68}. However, the tool is not executed. For that to happen we need an agent execution loop, which will handle the multiple iterations of generation to tool calling to generation, etc.

We use the AgentExecutor class to handle the execution loop:

In [18]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    memory = memory, 
    verbose = True
)

Now let's try the same query with the executor, note that the intermediate_steps parameter that we added before is no longer needed as the executor handles it internally.

In [19]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.1759999999999910.7 × 7.68 = 82.176 (rounded to three decimal places).

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 × 7.68\u202f=\u202f82.176 (rounded to three decimal places).', additional_kwargs={}, response_metadata={})],
 'output': '10.7 × 7.68\u202f=\u202f82.176 (rounded to three decimal places).'}

In [20]:
10.7*7.68

82.17599999999999

Let's test our agent with some memory and tool use. First, we tell it our name, then we will perform a few tool calls, then see if the agent can still recall our name.

First, give the agent our name:

In [21]:
agent_executor.invoke({
    "input": "My name is Ankur",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Nice to meet you, Ankur! How can I help you today?

> Finished chain.


{'input': 'My name is Ankur',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 × 7.68\u202f=\u202f82.176 (rounded to three decimal places).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Ankur', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ankur! How can I help you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, Ankur! How can I help you today?'}

In [22]:
agent_executor.invoke({
    "input": "What is 9+10-(4x2)^3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
9 + 10 − (4 × 2)³  
= 19 − 8³  
= 19 − 512  
= **−493**

> Finished chain.


{'input': 'What is 9+10-(4x2)^3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 × 7.68\u202f=\u202f82.176 (rounded to three decimal places).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Ankur', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ankur! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is 9+10-(4x2)^3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='9\u202f+\u202f10\u202f−\u202f(4\u202f×\u202f2)³  \n=\u202f19\u202f−\u202f8³  \n=\u202f19\u202f−\u202f512  \n= **−493**', additional_kwargs={}, response_metadata={})],
 'output': '9\u202f+\u202f10\u202f−\u202f(4\u202f×\u202f2)³  \n=\u202f19\u202f−\u202f8³  \n=\u202f19\u202f−\u202f512  \n= **−493**'}

In [23]:
9+10-(4*2)**3

-493

In [24]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Your name is Ankur.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 × 7.68\u202f=\u202f82.176 (rounded to three decimal places).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Ankur', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ankur! How can I help you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is 9+10-(4x2)^3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='9\u202f+\u202f10\u202f−\u202f(4\u202f×\u202f2)³  \n=\u202f19\u202f−\u202f8³  \n=\u202f19\u202f−\u202f512  \n= **−493**', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ankur.', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Ankur.'}

### SerpAPI Weather Agent
In this example, we'll be using the same agent and executor setup as before, but we'll be adding the SerpAPI service to allow our agent to search the web for information.

To use this tool, you need an API key, with the free plan you can use up to 250 searches per month.

In [25]:
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [26]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names = ['serpapi'], llm = llm)

These custom tools can look into your IP address, find out where you are currently, then we will also use a secondary function to get the current date and time, then we will use this information to feed into the SerpAPI to find us the weather pattern in your area and at the time of the function calling.

In [35]:
# defining a tool function that will retrieve the location from your ip

import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """
    Gets the Current Location based on the Ip Address.
    """
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                        f"Latitude: {latitude},\n"
                        f"Longitude: {longitude},\n"
                        f"City: {data.get('city','N/A')},\n"
                        f"Country: {data.get('country','N/A')}"
            )
            print(data)
            return data
        else:
            "Location could not be determined"
    except Exception as err:
        print(err)

@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

We can create our prompt, this time we'll skip the chat_history part as we don't need it. However, you can add it if preferred.

In [32]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

Now we create our full tools list, our agent, and the agent_executor:

In [36]:
tools = toolbox + [get_location_from_ip, get_current_datetime]

agent = create_tool_calling_agent(
    llm = llm,
    tools = tools,
    prompt = prompt
)

agent_executor = AgentExecutor(
    agent= agent,
    tools = tools,
    verbose = True
)

In [37]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_location_from_ip` with `{}`


Latitude: 28.6519,
Longitude: 77.2315,
City: Delhi,
Country: IN
Latitude: 28.6519,
Longitude: 77.2315,
City: Delhi,
Country: IN
Invoking: `Search` with `current weather in Delhi Celsius`


{'type': 'weather_result', 'temperature': '32', 'unit': 'Celsius', 'precipitation': '0%', 'humidity': '67%', 'wind': '5 km/h', 'location': 'Delhi, India', 'date': 'Tuesday 11:00 AM', 'weather': 'Partly cloudy'}
Invoking: `get_current_datetime` with `{}`


2025-09-30 11:03:01**Current date & time**  
2025‑09‑30 11:03 AM (local time)

**Weather in Delhi (approximate, as of the latest available data)**  
- **Temperature:** 32 °C  
- **Condition:** Partly cloudy  
- **Humidity:** 67 %  
- **Wind:** 5 km/h  
- **Precipitation:** 0 %

*(All values are from the most recent weather snapshot and may vary slightly by the time you check.)*

> Finished chain.


In [39]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, How is the weather  n Ludhiana, Punjab, India? Please give me degrees in Celsius"
    )
})



> Entering new AgentExecutor chain...

Invoking: `Search` with `weather Ludhiana Punjab India`


{'type': 'weather_result', 'temperature': '93', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '52%', 'wind': '2 mph', 'location': 'Ludhiana, Punjab, India', 'date': 'Tuesday 11:00 AM', 'weather': 'Sunny'}Here’s the current weather snapshot for **Ludhiana, Punjab, India** (as of the latest update):

| Item | Value |
|------|-------|
| **Temperature** | **≈ 34 °C** (93 °F) |
| **Humidity** | 52 % |
| **Wind** | 2 mph (≈ 3.2 km/h) |
| **Precipitation** | 0 % (no rain) |
| **Weather** | Sunny |
| **Time** | Tuesday, 11:00 AM (local time) |

**Quick note on the conversion:**  
93 °F → (93 – 32) × 5/9 ≈ 61 × 5/9 ≈ **33.9 °C**, which rounds to **34 °C**.

Feel free to let me know if you’d like more detailed forecasts or historical data!

> Finished chain.


In [40]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))

Here’s the current weather snapshot for **Ludhiana, Punjab, India** (as of the latest update):

| Item | Value |
|------|-------|
| **Temperature** | **≈ 34 °C** (93 °F) |
| **Humidity** | 52 % |
| **Wind** | 2 mph (≈ 3.2 km/h) |
| **Precipitation** | 0 % (no rain) |
| **Weather** | Sunny |
| **Time** | Tuesday, 11:00 AM (local time) |

**Quick note on the conversion:**  
93 °F → (93 – 32) × 5/9 ≈ 61 × 5/9 ≈ **33.9 °C**, which rounds to **34 °C**.

Feel free to let me know if you’d like more detailed forecasts or historical data!